In [12]:
import datetime
import os
from typing import Callable, Optional
import pandas as pd
from sklearn import preprocessing
import numpy as np
import torch
import networkx as nx
# import holoviews as hv
# from holoviews import opts
# hv.extension('bokeh')
import bokeh as bk

from torch_geometric.data import (
    Data,
    InMemoryDataset
)

pd.set_option('display.max_columns', None)
path = './IBM AML/data/raw/HI-Small_Trans.csv'
# path = '/media/data2/drop/HI-Small_Trans.csv'
df = pd.read_csv(path)

In [13]:
df.head(10)

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0
5,2022/09/01 00:03,1,8000F5AD0,1,8000F5AD0,6162.44,US Dollar,6162.44,US Dollar,Reinvestment,0
6,2022/09/01 00:08,1,8000EBAC0,1,8000EBAC0,14.26,US Dollar,14.26,US Dollar,Reinvestment,0
7,2022/09/01 00:16,1,8000EC1E0,1,8000EC1E0,11.86,US Dollar,11.86,US Dollar,Reinvestment,0
8,2022/09/01 00:26,12,8000EC280,2439,8017BF800,7.66,US Dollar,7.66,US Dollar,Credit Card,0
9,2022/09/01 00:21,1,8000EDEC0,211050,80AEF5310,383.71,US Dollar,383.71,US Dollar,Credit Card,0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078345 entries, 0 to 5078344
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Timestamp           object 
 1   From Bank           int64  
 2   Account             object 
 3   To Bank             int64  
 4   Account.1           object 
 5   Amount Received     float64
 6   Receiving Currency  object 
 7   Amount Paid         float64
 8   Payment Currency    object 
 9   Payment Format      object 
 10  Is Laundering       int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 426.2+ MB


In [15]:
df['id_from'] = df['From Bank'].astype(str) + '_' + df['Account']
df['id_to'] = df['To Bank'].astype(str) + '_' + df['Account.1']

In [16]:
df['id_from'].nunique(), df['id_to'].nunique()

(496999, 420640)

In [17]:
unique_values = pd.concat([df['id_from'], df['id_to']])
df_node = pd.DataFrame(unique_values.unique(), columns=['ID'])
df_node.head(10)

,ID
0,10_8000EBD30
1,3208_8000F4580
2,3209_8000F4670
3,12_8000F5030
4,10_8000F5200
5,1_8000F5AD0
6,1_8000EBAC0
7,1_8000EC1E0
8,12_8000EC280
9,1_8000EDEC0


In [18]:
df_edge = df.drop(columns=[
    'From Bank',
    'Account',
    'To Bank',
    'Account.1'
])

df_edge.head(10)

,Timestamp,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,id_from,id_to
0,2022/09/01 00:20,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0,10_8000EBD30,10_8000EBD30
1,2022/09/01 00:20,0.01,US Dollar,0.01,US Dollar,Cheque,0,3208_8000F4580,1_8000F5340
2,2022/09/01 00:00,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0,3209_8000F4670,3209_8000F4670
3,2022/09/01 00:02,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0,12_8000F5030,12_8000F5030
4,2022/09/01 00:06,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0,10_8000F5200,10_8000F5200
5,2022/09/01 00:03,6162.44,US Dollar,6162.44,US Dollar,Reinvestment,0,1_8000F5AD0,1_8000F5AD0
6,2022/09/01 00:08,14.26,US Dollar,14.26,US Dollar,Reinvestment,0,1_8000EBAC0,1_8000EBAC0
7,2022/09/01 00:16,11.86,US Dollar,11.86,US Dollar,Reinvestment,0,1_8000EC1E0,1_8000EC1E0
8,2022/09/01 00:26,7.66,US Dollar,7.66,US Dollar,Credit Card,0,12_8000EC280,2439_8017BF800
9,2022/09/01 00:21,383.71,US Dollar,383.71,US Dollar,Credit Card,0,1_8000EDEC0,211050_80AEF5310


In [19]:
df_edge[df_edge['Is Laundering']==1]

,Timestamp,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,id_from,id_to
4742,2022/09/01 00:21,389769.39,US Dollar,389769.39,US Dollar,Cheque,1,70_100428660,1124_800825340
40621,2022/09/01 00:03,29024.33,US Dollar,29024.33,US Dollar,Credit Card,1,70_100428660,11474_805B716C0
69152,2022/09/01 00:01,792.92,US Dollar,792.92,US Dollar,Credit Card,1,70_100428660,15980_80B39E7B0
85763,2022/09/01 00:03,13171425.53,US Dollar,13171425.53,US Dollar,Cheque,1,70_100428660,113798_80DC756E0
90684,2022/09/01 00:23,14288.83,US Dollar,14288.83,US Dollar,Cash,1,70_100428660,32375_80E480620
...,...,...,...,...,...,...,...,...,...
5071168,2022/09/10 23:29,7773.42,US Dollar,7773.42,US Dollar,ACH,1,111433_8065C2380,21550_8065C69E0
5073375,2022/09/10 23:07,448920.65,Ruble,448920.65,Ruble,Cheque,1,9_80727F930,219557_8072A28C0
5074248,2022/09/12 03:34,20085.71,Saudi Riyal,20085.71,Saudi Riyal,ACH,1,222_812D127D0,50202_812D129C0
5075056,2022/09/10 23:43,3314.36,US Dollar,3314.36,US Dollar,ACH,1,17327_8095F6C20,15695_8095F6C70


In [44]:

# Membuat graph dari DataFrame
grp = nx.from_pandas_edgelist(df_edge[df_edge['Is Laundering']==1], 'id_from', 'id_to', [
    'Amount Received', 'Receiving Currency', 'Amount Paid',
    'Payment Currency', 'Payment Format', 'Is Laundering',
])

bk_graph = bk.plotting.from_networkx(grp, nx.spring_layout, scale=1, center=(0, 0))

# Membuat plot figure
fig = bk.models.Plot(
    width=800, height=800,
    x_range=bk.models.Range1d(-1.1, 1.1), 
    y_range=bk.models.Range1d(-1.1, 1.1)
)

# Menambahkan tools interaktif termasuk alat zoom
fig.add_tools(
    bk.models.HoverTool(tooltips=[
        ("index", "$index"), 
        ("Amount Paid", "@{Amount Paid}"),
        ("Amount Received", "@{Amount Received}")
    ]), 
    bk.models.TapTool(), 
    bk.models.BoxSelectTool(),
    bk.models.WheelZoomTool()
)
fig.toolbar.active_scroll = fig.select_one(bk.models.WheelZoomTool)

# Menambahkan styling
bk_graph.node_renderer.glyph = bk.models.Circle(
    radius=0.01, 
    fill_color='skyblue'
)
bk_graph.edge_renderer.glyph = bk.models.MultiLine(
    line_color="#CCCCCC", 
    line_alpha=0.8, 
    line_width=1
)

# append graph to plot 
fig.renderers.append(bk_graph)

# Menyimpan plot ke file HTML
bk.io.output_file("graph.html")
bk.io.save(fig)

'/Users/mohzulkiflikatili/2latihan_python/BI_AI_FINNANCE/notebook/graph.html'

In [ ]:
cek = df.groupby(['id_to']).agg({
    'id_from':'nunique',
    'Is Laundering':'first',
    'Amount Paid':'sum',
}).reset_index().sort_values(['Amount Paid'], ascending=False)

cek

In [10]:
def find_parameters_and_result(numbers):
    length = len(numbers)
    for i in range(length):
        sum_rest = sum(numbers) - numbers[i]
        if sum_rest == numbers[i]:
            result = numbers[i]
            parameters = numbers[:i] + numbers[i+1:]
            return parameters, result
    return None, None

# Contoh penggunaan
numbers = [19, 9, 10]
parameters, result = find_parameters_and_result(numbers)
print("Parameters:", parameters)
print("Result:", result)

Parameters: [9, 10]
Result: 19


In [55]:

df['Group'] = df.groupby('Timestamp').ngroup()
df_sorted = df.sort_values(by='Group')
df_sorted[
    (df_sorted['Group']==3) & 
    (df_sorted['Is Laundering']==1)
]


,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,id_from,id_to,Group
85763,2022/09/01 00:03,70,100428660,113798,80DC756E0,13171425.53,US Dollar,13171425.53,US Dollar,Cheque,1,70_100428660,113798_80DC756E0,3
40621,2022/09/01 00:03,70,100428660,11474,805B716C0,29024.33,US Dollar,29024.33,US Dollar,Credit Card,1,70_100428660,11474_805B716C0,3
317686,2022/09/01 00:03,1467,8013C4030,20,80BC62F10,58702.10,Yuan,58702.10,Yuan,ACH,1,1467_8013C4030,20_80BC62F10,3


In [57]:
8105.79+29024.33

37130.12

# Bokeh

In [4]:
# import pandas as pd
# import networkx as nx
# from bokeh.io import show, output_file, save
# from bokeh.plotting import from_networkx
# from bokeh.models import Plot, Range1d, Circle, MultiLine, HoverTool, TapTool, BoxSelectTool

# # Contoh DataFrame dengan atribut edge
# data = {'source': ['A', 'A', 'B', 'C', 'C', 'D'],
#         'target': ['B', 'C', 'D', 'D', 'E', 'E'],
#         'weight': [1, 2, 3, 4, 5, 6],
#         'label': ['ab', 'ac', 'bd', 'cd', 'ce', 'de']}
# df = pd.DataFrame(data)

# # Membuat graf dari DataFrame dengan atribut edge
# G = nx.from_pandas_edgelist(df, 'source', 'target', edge_attr=True)

# # Membuat plot Bokeh
# plot = Plot(
#         width=800, 
#         height=800,
#         x_range=Range1d(-1.1, 1.1), 
#         y_range=Range1d(-1.1, 1.1)
# )

# # Menambahkan tools interaktif
# plot.add_tools(
#     HoverTool(tooltips=[("index", "$index"), ("(x,y)", "($x, $y)")]),
#     TapTool(), BoxSelectTool()
# )

# # Membuat layout node
# graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))

# # Menambahkan styling
# graph_renderer.node_renderer.glyph = Circle(radius=00.1, fill_color='skyblue')
# graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=1)

# plot.renderers.append(graph_renderer)

# # Menyimpan plot ke file HTML
# output_file("graph.html")
# save(plot)

# # Menampilkan plot
# # show(plot)

'/Users/mohzulkiflikatili/2latihan_python/BI_AI_FINNANCE/notebook/graph.html'